# hot-CNO and breakout

In [1]:
import pynucastro as pyna

This collection of rates has the main CNO rates plus a breakout rate into the hot CNO cycle

In [2]:
rl = pyna.ReacLibLibrary()

In [3]:
rate_names = ["c12(p,g)n13",
              "c13(p,g)n14",
              "n13(,)c13",
              "n13(p,g)o14",
              "n14(p,g)o15",
              "n15(p,a)c12",
              "o14(,)n14",
              "o15(,)n15",
              "o14(a,p)f17",
              "f17(p,g)ne18",
              "ne18(,)f18",
              "f18(p,a)o15"]

In [4]:
rates = rl.get_rate_by_name(rate_names)

In [5]:
rc = pyna.RateCollection(rates=rates)

To evaluate the rates, we need a composition.  This is defined using a list of Nuceli objects.  

In [6]:
comp = pyna.Composition(rc.get_nuclei())
comp.set_solar_like()

Interactive exploration is enabled through the Explorer class, which takes a RateCollection and a Composition

In [7]:
re = pyna.Explorer(rc, comp, size=(1000,1000),
                   ydot_cutoff_value=1.e-25)
re.explore()

interactive(children=(FloatSlider(value=4.0, description='logrho', max=6.0, min=2.0), FloatSlider(value=8.0, d…